In [ ]:
#default_exp tabular.data

In [ ]:
#export
from fastai2.torch_basics import *
from fastai2.data.all import *
from fastai2.tabular.core import *

## TabularDataLoaders -

In [ ]:
#export
class TabularDataLoaders(DataLoaders):
    @classmethod
    @delegates(Tabular.dataloaders)
    def from_df(cls, df, path='.', procs=None, cat_names=None, cont_names=None, y_names=None, block_y=None,
                valid_idx=None, **kwargs):
        if cat_names is None: cat_names = []
        if cont_names is None: cont_names = list(set(df)-set(cat_names)-set(y_names))
        splits = RandomSplitter()(df) if valid_idx is None else IndexSplitter(valid_idx)(df)
        to = TabularPandas(df, procs, cat_names, cont_names, y_names, splits=splits, block_y=block_y)
        return to.dataloaders(path=path, **kwargs)

    @classmethod
    @delegates(Tabular.dataloaders)
    def from_csv(cls, csv, path='.', procs=None, cat_names=None, cont_names=None, y_names=None, block_y=None,
                valid_idx=None, **kwargs):
        return cls.from_df(pd.read_csv(csv), path, procs, cat_names=cat_names, cont_names=cont_names, y_names=y_names,
                           block_y=block_y, valid_idx=valid_idx, **kwargs)
    
    @delegates(TabDataLoader.__init__)
    def test_dl(self, test_items, rm_type_tfms=None, **kwargs):
        to = self.train_ds.new(test_items)
        to.process()
        return self.valid.new(to, **kwargs)
    
Tabular._dbunch_type = TabularDataLoaders

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_main,df_test = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_main.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
dls = TabularDataLoaders.from_df(df, path, procs, cat_names, cont_names, y_names="salary",
                                 valid_idx=list(range(800,1000)), bs=64)

In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Separated,Transport-moving,Unmarried,Black,False,False,False,40.0,210275.000031,9.0,<50k
1,Private,HS-grad,Never-married,Machine-op-inspct,Other-relative,White,False,False,False,20.0,116790.999251,9.0,<50k
2,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,False,False,30.0,154950.000888,10.0,>=50k
3,Private,HS-grad,Separated,Craft-repair,Unmarried,White,False,False,False,31.0,442428.988418,9.0,<50k
4,Local-gov,10th,Never-married,Other-service,Own-child,White,False,False,False,17.0,39815.001024,6.0,<50k
5,Self-emp-not-inc,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,60.0,282660.000736,4.0,<50k
6,Private,HS-grad,Never-married,Craft-repair,Not-in-family,Black,False,False,False,27.0,287475.998973,9.0,<50k
7,Private,Masters,Married-civ-spouse,Exec-managerial,Husband,White,False,False,False,41.0,101592.996916,14.0,>=50k
8,Self-emp-not-inc,Bachelors,Married-civ-spouse,Sales,Husband,White,False,False,False,33.0,312055.006129,13.0,>=50k
9,Self-emp-not-inc,5th-6th,Married-civ-spouse,Farming-fishing,Husband,White,False,False,False,62.0,215943.999918,3.0,<50k


In [ ]:
type(dls)

__main__.TabularDataLoaders

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.learner.ipynb.
Converted 22_tutorial.imagenette.ipynb.
Converted 23_tutoria